In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Real-time valuation using streamed market data

This notebook shows how to value a portfolio using real-time streamed market data

Attributes
----------
valuation
recipes
"""

toggle_code("Hide docstring")

# Real-time Valuation

## Introduction

This notebook demonstrates how to configure LUSID to perform a real-time [*GetValuation*](https://www.lusid.com/docs/api/#operation/GetValuation) call.

### Instrument universe

This notebook requires an instrument universe to be populated in a file at the location `data/real-time-valuation/SIX-CROSSREFERENCE.csv`.  The `template` file in the same directory indicates the required columns.  The easiest way to create this file is using the CROSSREFERENCE dataset provided by SIX.  This can be accessed using Luminesce using a query like below in a LUSID environment which is entitled to access SIX data.

For each instrument you will need a identifier and the BC (exchange) code.

```
values
  ('1222171'),                -- A set of instrument identifiers
  ('2340545');

@dataQuery = select
  'CH' AS identifierScheme,  -- SIX-specific identifier type code
  column1 AS identifier,
  '4' as bc                  -- SIX-specific exchange code
from @data;

Select * from Six.Flex where IncludeDataSetId = TRUE
AND IncludeStatus = TRUE
AND PackageId = 'CrossReference'
AND Request = @dataQuery;

```

In [2]:
# Import system packages
import os
import pandas as pd
import random
from datetime import timedelta
from IPython.core.display import HTML


# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python

import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")
api_url = api_factory.api_client.configuration._base_path.replace("api","")
print ('LUSID Environment :', api_url)

LUSID Environment : https://steco.lusid.com/


In [3]:
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
aggregation_api = api_factory.build(lusid.AggregationApi)
quotes_api = api_factory.build(lusid.QuotesApi)
instruments_api = api_factory.build(lusid.InstrumentsApi)

In [4]:
scope = "realtime-valuation"
portfolio_code = "portfolio"
recipe_code = "SixStreaming-Valoren"
default_currency = "GBP"

## 1. Setup

The SIX realtime pricing feed relies on at least one of SIX-specific instrument identifiers being configured in your LUSID instance

In [5]:
required_id_types = set([
    'SixIsin_BC',
    'SixValoren_BC',
    'SixSedol_BC',
    'SixCusip_BC',
    'SixTicker_BC'
])

instrument_id_types = instruments_api.get_instrument_identifier_types()
domain_id_types = set([id.identifier_type for id in instrument_id_types.values if id.identifier_type.endswith("_BC")])

missing_id_types = required_id_types - domain_id_types

if len(missing_id_types) > 0:
    print(f"The following SIX Instrument identifier types need to be setup in {api_url}: {missing_id_types}")

## 2. Load Data

### 2.1 Instruments 

In [ ]:
# Read a standard SIX CROSSREFERENCE file
instruments_df = pd.read_csv("data/real-time-valuation/SIX-CROSSREFERENCE.csv")

# Add required columns
instruments_df['ISIN_BC']= instruments_df['ISIN'].astype(str) + '_'+ instruments_df['BC'].astype(str)
instruments_df['tradingSymbol_BC']= instruments_df['tradingSymbol'].astype(str) + '_'+ instruments_df['BC'].astype(str)
instruments_df['swissValorNumber_BC']= instruments_df['swissValorNumber'].astype(str) + '_'+ instruments_df['BC'].astype(str)

instruments_df

Create a mapping schema for the instruments using the provided FIGIs as the instrument identifiers. The instruments file is loaded into LUSID. 

In [7]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "swissValorNumber",
        "Figi" : "FIGIGlobalShareClassId",
        "Isin" : "ISIN",
        "SixIsin_BC" : "ISIN_BC",
        "Ticker" : "tradingSymbol",
        "SixTicker_BC" : "tradingSymbol_BC",
        "SixValoren_BC" : "swissValorNumber_BC"
    },
    "required": {
        "name": "FISNSIX"
    },
}

# Instruments can be loaded using a dataframe with file_type set to "instruments"
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instruments_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

#result

/opt/conda/lib/python3.11/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


,success,failed,errors
0,31,0,0


The instruments should now be viewable in the [LUSID webtool](https://www.lusid.com/app/home) (*Data Management* >>> *Instruments*)

### 1.2 Portfolio

Create a portfolio and populate with simulated transactions for the Instruments loaded above

In [8]:
# A portfolio can be loaded using a dataframe with file_type = "portfolios"
portfolio_mapping = {
    "required": {
        "code": f"${portfolio_code}",
        "display_name": "$Realtime Test Portfolio",
        "base_currency": f"${default_currency}",
    },
    "optional": {"created": "$2020-01-01T00:00:00+00:00"},
}

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=pd.DataFrame(['1']),
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[],
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

/opt/conda/lib/python3.11/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


,success,failed,errors
0,1,0,0


### 1.3 Holdings

In [9]:
# Here we generate a Holdings dataset from the Instrument universe with fake holdings
# For simplicity, always use the same set of random numbers
rand = random.Random()
rand.seed("test")

holdings_df = instruments_df[['swissValorNumber']].copy()
holdings_df['swissValorNumber'] = holdings_df['swissValorNumber'].astype(str)
holdings_df['units'] = [rand.randrange(100,1000) for r in range(0,len(holdings_df))]
holdings_df['cost'] = [rand.randrange(1000,10000) for r in range(0,len(holdings_df))]
holdings_df['price'] = holdings_df['cost'] / holdings_df['units']
holdings_df.head()

,swissValorNumber,units,cost,price
0,22627015,771,1354,1.76
1,20100637,958,4803,5.01
2,52755147,852,2309,2.71
3,24275396,982,9197,9.37
4,10299318,356,9306,26.14


In [10]:
holdings_mapping = {
    "identifier_mapping": {
        "ClientInternal": "swissValorNumber"
    },
    "required": {
        "effective_at": "$2020-01-01",
        "code": f"${portfolio_code}",
        "tax_lots.units": "units",
        "tax_lots.portfolio_cost": "cost",
        "tax_lots.cost.currency": f"${default_currency}",
        "tax_lots.cost.amount": "cost"
    },
    "optional": {}
}

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=holdings_df,
    mapping_required=holdings_mapping["required"],
    mapping_optional=holdings_mapping["optional"],
    file_type="holdings",
    identifier_mapping=holdings_mapping["identifier_mapping"],
)

#succ, failed = format_holdings_response(result)
#display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

/opt/conda/lib/python3.11/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


### 1.4 Prices

In [11]:
quotes_supplier = "Lusid"
quotes_field = "mid"
quotes_instrument_id = "ClientInternal"

quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": f"${quotes_instrument_id}",
    "quote_id.effective_at": "$2024-02-01",
    "quote_id.quote_series_id.provider": f"${quotes_supplier}",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "swissValorNumber",
    "quote_id.quote_series_id.field" : f"${quotes_field}",
    "metric_value.unit": f"${default_currency}",
    "metric_value.value": "price"
}

result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=holdings_df,
        mapping_required=quotes_mapping,
        mapping_optional={},
        file_type="quotes",
)

succ, failed, errors = format_quotes_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

/opt/conda/lib/python3.11/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


,success,failed,errors
0,31,0,0


### 1.5 Exchange Rates

In [12]:
# Create dummy rates from GBP and CHF
rates = [['GBP/USD', 1.26], ['USD/CHF', 0.87]]
 
instrument_id_type = "CurrencyPair"
value = "Rate"

rates_df = pd.DataFrame(rates, columns=[instrument_id_type, value])
 
rates_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$CurrencyPair",
    "quote_id.effective_at": "$2024-02-01",
    "quote_id.quote_series_id.provider": f"${quotes_supplier}",
    "quote_id.quote_series_id.quote_type": "$Rate",
    "quote_id.quote_series_id.instrument_id": instrument_id_type,
    "quote_id.quote_series_id.field" : f"${quotes_field}",
    "metric_value.unit": instrument_id_type,
    "metric_value.value": value
}


result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=rates_df,
        mapping_required=rates_mapping,
        mapping_optional={},
        file_type="quotes",
)

succ, failed, errors = format_quotes_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

/opt/conda/lib/python3.11/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


,success,failed,errors
0,2,0,0


## 2. Run valuation

In [13]:
configuration_recipe = models.ConfigurationRecipe(
        scope=scope,
        code=recipe_code,
        market=models.MarketContext(
            market_rules=[
                # First source is real-time
                models.MarketDataKeyRule(
                    key="Quote.SixValoren_BC.*",
                    supplier="SIX",
                    quote_type="Price",
                    data_scope="NotUsed",
                    field="Last",
                    price_source="",
                    source_system="SIX/Streaming"
                ),
                # Fall back to stored quotes
                models.MarketDataKeyRule(
                    key="Quote.ClientInternal.*",
                    supplier=quotes_supplier,
                    quote_type="Price",
                    data_scope=scope,
                    field=quotes_field,
                    quote_interval="1Y.0D"
                ),
                # 
                models.MarketDataKeyRule(
                    key='Fx.CurrencyPair.*',
                    data_scope=scope,
                    supplier='Lusid',
                    quote_type='Rate',
                    quote_interval='1Y.0D',
                    field="mid"
                )
            ],
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="LusidInstrumentId",
                default_scope=scope,
                attempt_to_infer_missing_fx=True
            ),
        ),
        pricing=models.PricingContext(
            options={"AllowPartiallySuccessfulEvaluation": True},
        ),
    )

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

In [14]:
# Pull the data aggregation by passing the effectiveAt date
def aggregation_request(effectiveAt):
    return models.ValuationRequest( 
        recipe_id = models.ResourceId(
            scope = scope,
            code = recipe_code
        ),
        metrics = [
            models.AggregateSpec("Instrument/default/Name", "Value"),
            models.AggregateSpec("Valuation/PV", "Proportion"),
            models.AggregateSpec("Valuation/PV", "Sum"),
            models.AggregateSpec("Holding/default/Units", "Sum"),
            models.AggregateSpec("Quotes/Price", "Value"),
            models.AggregateSpec("Quotes/Price/EffectiveAt", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        # choose the valuation date for the request - set using effectiveAt
        valuation_schedule=models.ValuationSchedule(effective_at=effectiveAt),
        portfolio_entity_ids = [models.PortfolioEntityId(
                                                        scope = scope,
                                                        code = portfolio_code,
                                                        portfolio_entity_type="SinglePortfolio" 
            )]
        )

aggregation = aggregation_api.get_valuation(
  valuation_request=aggregation_request("2020-08-24T01:01:00.000Z")
)
pd.DataFrame(aggregation.data)

,Instrument/default/Name,Proportion(Valuation/PV),Sum(Valuation/PV),Sum(Holding/default/Units),Quotes/Price,Quotes/Price/EffectiveAt,Aggregation/Errors
0,Allegion/RegSh USD0.01,None,"99,528.39",771.00,129.09,2024-02-23T21:15:00.0000000+00:00,"[metadata mismatch Dim:0,Units:(CHF) != Dim:0,..."
1,Eaton Corp/RegSh USD0,None,"272,962.94",958.00,284.93,2024-02-23T21:15:00.0000000+00:00,"[metadata mismatch Dim:0,Units:(CHF) != Dim:0,..."
2,Trane Tech/RegSh USD1,None,"240,110.64",852.00,281.82,2024-02-23T21:15:00.0000000+00:00,"[metadata mismatch Dim:0,Units:(CHF) != Dim:0,..."
3,Pentair/RegSh USD0.01,None,"75,378.32",982.00,76.76,2024-02-23T21:15:00.0000000+00:00,"[metadata mismatch Dim:0,Units:(CHF) != Dim:0,..."
4,TE Connectiv/RegSh CHF0.57,None,"50,947.16",356.00,143.11,2024-02-23T21:15:00.0000000+00:00,"[metadata mismatch Dim:0,Units:(CHF) != Dim:0,..."
5,Accenture/RegSh Cl-A USD0.0000225,None,"282,990.00",750.00,377.32,2024-02-23T21:15:00.0000000+00:00,"[metadata mismatch Dim:0,Units:(CHF) != Dim:0,..."
6,Johnson Ctr Int/RegSh USD0.01,None,"51,251.88",877.00,58.44,2024-02-23T21:15:00.0000000+00:00,"[metadata mismatch Dim:0,Units:(CHF) != Dim:0,..."
7,Amcor/RegSh USD0.01,None,"3,151.80",340.00,9.27,2024-02-23T21:15:00.0000000+00:00,"[metadata mismatch Dim:0,Units:(CHF) != Dim:0,..."
8,Aptiv/RegSh USD0.01,None,"22,551.16",292.00,77.23,2024-02-23T21:15:00.0000000+00:00,"[metadata mismatch Dim:0,Units:(CHF) != Dim:0,..."
9,STERIS/RegSh USD0.001,None,"33,254.85",141.00,235.85,2024-02-23T21:15:01.0000000+00:00,"[metadata mismatch Dim:0,Units:(CHF) != Dim:0,..."


In [15]:
display(HTML("<h1>Links</h1>"))

display(HTML(f'''
  <a href="{api_url}app/dashboard/holdings?scope={scope}&code={portfolio_code}&entityType=Portfolio&recipeScope={scope}&recipeCode={recipe_code}"
  target="_blank">
    Holdings
  </a>'''))

display(HTML(f'''
  <a href="{api_url}app/dashboard/valuations?scope={scope}&code={portfolio_code}&entityType=Portfolio&recipeScope={scope}&recipeCode={recipe_code}"
  target="_blank">
    Valuation with real-time prices
  </a>'''))